In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Green Suppliers

In [7]:
df_green = pd.read_csv('./관련자료/green.csv')
df_green.head()

,suppliers,energy_consumption,sales,co2_emission,R&D
0,S1,1267,119477,43562,67
1,S2,968,125762,45000,65
2,S3,1001,58770,42400,57
3,S4,1393,62989,43734,62
4,S5,1586,67088,44890,50


In [10]:
for row in df_green:
    print(row)

suppliers
energy_consumption
sales
co2_emission
R&D


In [4]:
# 평가를 위한 사전 설정?
# >> 최댓값, 최솟값을 어떻게 설정할지 <<

cols = df_green.columns[1:]
energy_option = {'type' : 'dae', 'max' : 1998, 'min' : 637, 'spread' : 50}
sales_option = {'type' : 'dae', 'max' : 125762, 'min' : 55681, 'spread' : 5000}
co2_option = {'type' : 'so', 'max' : 45023, 'min' : 3870, 'spread' : 500}
RnD_option = {'type' : 'dae', 'max' : 570, 'min' : 36, 'spread' : 0.5}

col_option = pd.DataFrame([energy_option, sales_option, co2_option, RnD_option])
col_option.index = cols
col_option

,type,max,min,spread
energy_consumption,dae,1998,637,50.0
sales,dae,125762,55681,5000.0
co2_emission,so,45023,3870,500.0
R&D,dae,570,36,0.5


In [5]:
def fuzzify_data(data):
    for col in data.columns:
        if col == 'suppliers':
            data_fuz = pd.DataFrame(data[col]).copy()
        else:
            spr = col_option.loc[col, 'spread']
            col_fuz = [(val-spr, val, val+spr) for val in data[col]]
            data_fuz[col] = col_fuz

    return data_fuz

df_fuz = fuzzify_data(df_green)
df_fuz.head()

,suppliers,energy_consumption,sales,co2_emission,R&D
0,S1,"(1217.0, 1267, 1317.0)","(114477.0, 119477, 124477.0)","(43062.0, 43562, 44062.0)","(66.5, 67, 67.5)"
1,S2,"(918.0, 968, 1018.0)","(120762.0, 125762, 130762.0)","(44500.0, 45000, 45500.0)","(64.5, 65, 65.5)"
2,S3,"(951.0, 1001, 1051.0)","(53770.0, 58770, 63770.0)","(41900.0, 42400, 42900.0)","(56.5, 57, 57.5)"
3,S4,"(1343.0, 1393, 1443.0)","(57989.0, 62989, 67989.0)","(43234.0, 43734, 44234.0)","(61.5, 62, 62.5)"
4,S5,"(1536.0, 1586, 1636.0)","(62088.0, 67088, 72088.0)","(44390.0, 44890, 45390.0)","(49.5, 50, 50.5)"


In [6]:
class model:
    def __init__(self):       
        pass    
    
    def col_score(self, inputs, col):
        y_i = inputs[col]
        
        # max, min을 따로 설정할 경우 구간 나눠야 함
        max_idx = np.argmax(df_green[col])
        min_idx = np.argmin(df_green[col])
        
        y_max = df_fuz[col][max_idx]
        y_min = df_fuz[col][min_idx]
        
        bunja = self.fuz_sub(y_i, y_min)
        bunmo = self.fuz_sub(y_max, y_min)
        d_i = self.fuz_div(bunja, bunmo)
        return d_i
    
    def score(self, inputs):
        scores = 0
        for col in inputs.index:
            if col == 'suppliers':
                continue
            
            d = self.col_score(inputs, col)
            scores += np.array(d)
        
        return scores / (len(inputs) - 1)              
     
    def fuz_sub(self, num1, num2):
        left = num1[0] - num2[2]
        mid = num1[1] - num2[1]
        right = num1[2] - num2[0]
        return left, mid, right
    
    
    def fuz_div(self, num1, num2):
        cases = [a/b for a in [num1[0], num1[2]] for b in [num2[0], num2[2]]]
        left = min(cases)
        mid = num1[1] / num2[1]
        right = max(cases)
        return left, mid, right

In [7]:
model = model()

In [8]:
def round_fuz(number):
    return f'({number[0]:.3f}, {number[1]:.3f}, {number[2]:.3f})'

In [9]:
df_result = df_fuz.iloc[:, 0:1].copy()
results = []
for i, sup in df_fuz.iterrows():
    D = model.score(sup)
    results.append(round_fuz(D))
    
df_result['score'] = results
df_result.head()

,suppliers,score
0,S1,"(0.407, 0.551, 0.742)"
1,S2,"(0.422, 0.573, 0.773)"
2,S3,"(0.121, 0.236, 0.378)"
3,S4,"(0.252, 0.377, 0.537)"
4,S5,"(0.334, 0.466, 0.640)"
